## Chapter 20. 속성 디스크립터

### 20.1 디스크립터 예: 속성 검증
19.4절 '프로퍼티 팩토리 구현하기'에서는 프로퍼티 함수를 사용하였다. 프로퍼티 함수는 고위함수로 접근자 함수를 매개변수화하고 storage_name과 같은 환경 변수를 클로저에 담아서 사용자 정의 프로퍼티 객체를 생성한다. 이와 동일한 문제를 객체지향방식으로 해결한 것이 디스크립터 클래스이다.

#### 20.1.1 LineItem 버전 #3: 간단한 디스크립터
\_\_get\_\_( ), \_\_set__( ), \_\_delete\_\_( ) 메서드를 구현하는 클래스가 디스크립터이다. 디스크립터는 클래스 객체를 다른 클래스의 클래스 속성으로 정의해서 사용한다. 우리는 Quantity 디스크립터 클래스를 생성하고 LineItem 클래스는 두 개의 Quantity 객체를 사용할 것이다. 하나는 weight 속성을 다른 하나는 price 속성을 관리하기 위해 사용한다.


주요 용어는 아래과 같다.

<dl>
    <p>
      <dt><b>디스크립터 클래스</b></dt>
      <dd>디스크립터 프로토콜을 구현하는 클래스. Quantity 클래스가 디스크립터 클래스이다.</dd>
    </p>
    <p>
      <dt><b>관리 대상 클래스</b></dt>
      <dd>디스크립터 객처를 클래스 속성으로 선언하는 클래스. LineItem 클래스가 관리 대상 클래스이다.</dd>
    </p>
    <p>
      <dt><b>디스크립터 객체</b></dt>
      <dd>관리 대상 클래스의 클래스 속성으로 선언된 디스크립터 클래스의 객체. [그림 20-1]에서 각각의 디스크립터 객체는 밑줄 친 이름을 가진 구성 화살표로 표현된다.(UML에서 밑줄 친 속성은 클래스 속성을 나타낸다.) 디스크립터 객체를 가진 LineItem 클래스 쪽에 검은 마름모가 연결된다.</dd>
    </p>
    <p>
      <dt><b>관리 대상 객체</b></dt>
      <dd>관리 대상 클래스의 객체. 이 예제에서는 LineItem 클래스의 객체들이 관리 대상 객체가 된다.(클래스 다이어그램에는 나타나 있지 않다.)</dd>
    </p>
    <p>
      <dt><b>저장소 속성</b></dt>
      <dd>관리 대상 객체 안의 관리 대상 속성값을 담을 속성. [그림 20-1]에서 LineItem 객체의 weight와 price 속성이 저장소 속성이다. 이들은 디스크립터 객체와는 별개의 속성으로 향상 클래스 속성이다.</dd>
    </p>
    <p>
      <dt><b>관리 대상 속성</b></dt>
      <dd>디스크립터 객체에 의해 관리되는 관리 대상 클래스 안의 공개 속성으로 이 속성의 값은 저장소 속성에 저장된다. 즉, 디스크립터 객체와 저장소 속성이 관리 대상 속성에 대한 기반을 제공한다.</dd>
    </p>
</dl>

In [1]:
""" [예제 20-1] bulkfood_v3.py: LineItem의 속성을 관리하는 Quantity 디스크립터 """

class Quantity: # 상속은 필요없다.
    def __init__(self, storage_name):
        self.storage_name = storage_name # 관리대상 객체에서 값을 보관할 속성의 이름이다.
    
    def __set__(self, instance, value):
        """
        관리 대상 속성에 값을 할당할 때 호출된다.
        - self : 디스크립트 객체, 여기서는 LineItem.weight, LineItem.price에 대입하는 객체
        - instance : 관리 대상 객체, 여기서는 LintItem
        - value : 할당할 값
        """
        print(repr(self))
        print(repr(instance))
        if value > 0:
            instance.__dict__[self.storage_name] = value # 관리 대상 객체의 __dict__를 직접 처리한다. 
                                                         # setattr() 내장함수를 사용하면 또 다시 __set__() 메서드가 호출되어 무한 재귀가 된다.
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

\_\_set\_\_( ) 함수에서 아래 코드 처럼 바꾸고 싶은 생각이 들 수도 있지만 잘못된 것이다.
```
1) instance.__dict__[self.storage_name] = value
2) self.__dict__[self.storage_name] = value
```
여기서 self는 디스크립터 객체로서, 관리 대상 클래스의 클래스 속성이다. 메모리에 수천 개의 LineItem 객체가 있더라도 디스크립터 객체는 LineItem.weight, LineItem.price, 단 두 개 밖에 없다. 따라서 디스크립터 객체에 저장하는 모든 것은 LineItem 클래스 속성이 되어, 모든 LineItem 객체가 공유한다.

[예제 20-1]의 단점은 관리 대상 클래스 본체에 디스크립터 객체를 생성할 때 속성명을 반복해야 한다는 것이다. 아래와 같으면 더 좋을 것이다. 
```
class LineItem:
    weight = Quantity()
    price = Quantity()
```
문제는 변수가 존재하기도 전에 할당문의 오른쪽이 실행된다는 것이다. 이때 Quantity 클래스 안에 있는 코드에서는 디스크립터 객체를 어떤 이름의 변수에 바인딩해야 할지 알 수 없다. 프로그래머가 코드를 복사 붙여 넣고 변수명을 바꾸지 않아서, price = Quantity('weight')와 같이 되는 문제를 해결하기 위해 그리 멋지지 않지만 쓸 만한 해결책을 다음 절에서 소개한다. 

#### 20.1.2 LineItem 버전 #4: 자동 저장소 속성명
디스크립터를 선언할 때 속성명을 반복 입력하지 않기 위해 각 Quantity 객체의 storage_name에 대한 고유한 문자열을 생성할 것이다. storage_name을 생성하기 위해 '\_Quantity#' 문자열 뒤에 정수를 연결한다. 새로운 Quality 디스크립터 객체가 클래스에 연결될 때마다 Quantity.\_\_counter 클래스 속성의 값이 증가한다. 해시기호(#)를 사용하면 올바른 파이썬 문법이 아니기 때문에 점 표기법으로 생성한 속성과 충돌되지 않도록 보장할 수 있다.

여기서는 instance.\_\_dict\_\_ 대신 getattr()와 setattr() 내장함수를 이용해서 값을 저장할 수 있었다. 관리 대상 속성과 저장소 속성의 이름이 다르기 때문에 저장소 속성에 getattr()을 호출하더라도 디스크립터를 실행하지 않으므로, [예제 20-1]의 무한 재귀가 일어나지 않는다.

In [3]:
""" [예제 20-2] bulkfood_v4.py: 각 Quantity 디스크립터는 고유한 storage_name을 가진다. """

class Quantity:
    __counter = 0 # Quantity 객체의 수를 센다.
    
    def __init__(self):
        cls = self.__class__  # Quantity 클래스에 대한 참조
        prefix = cls.__name__ 
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) # 고유한 이름 (예를 들면_Quantity#0)을 만든다. 
        cls.__counter += 1
    
    def __get__(self, instance, owner): # 관리 대상 속성의 이름이 storage_name과 동일하지 않으므로 __get__() 메서드를 구현현다.
        return getattr(instance, self.storage_name) # instance에서 값을 가져온다.
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(, self.storage_name, value)
        elsinstancee:
            raise ValueError('value must be 0')

SyntaxError: invalid syntax (<ipython-input-3-6f46929e1360>, line 18)

In [4]:
class LineItem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weigth * self.price

TypeError: __init__() missing 1 required positional argument: 'storage_name'

In [5]:
coconuts = LineItem('Brazilian', 20, 17.95)

In [6]:
coconuts.weight, coconuts.price

(20, 17.95)

In [7]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

AttributeError: 'LineItem' object has no attribute '_Quantity#0'

In [8]:
LineItem.weight

\_\_get\_\_() 함수의 owner 인수는 관리 대상 클래스(즉, LienItem)에 대한 참조이며 디스크립터를 이용해서 클래스 속성을 가져올 때 유용하게 사용할 수 있다. LineItem.weight 처럼 글래수에서 관리 대상 속성을 가져올 때는 디스크립터 \_\_get\_\_() 메서드가 instance 인수값으로 None을 받는다. 따라서 위처럼 Nonetype과 _Quantity#0 등 내부 구현에 관련된 혼란스러운 메시지는 'LineItem' class has no such attribute 정도로 조절하는 것도 좋다. 

하지만 사용자가 내부 조사나 여타 메타프로그래밍 기법을 사용할 수 있도록 지원하려면, 클래스를 통해 관리 대상 속성에 접근할 때 \_\_get\_\_() 메서드가 디스크립터 객체를 반환하게 하는 것이 좋다. 아래는 이를 적용한 것이다.

In [9]:
""" [예제 20-3] bulkfood_v4.py: 관리되는 클래스를 통해 호출되면 get()은 디스크립터 자체에 대한 참조를 반환한다. """

class Quantity:
    __counter = 0 # Quantity 객체의 수를 센다.
    
    def __init__(self):
        cls = self.__class__  # Quantity 클래스에 대한 참조
        prefix = cls.__name__ 
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) # 고유한 이름 (예를 들면_Quantity#0)을 만든다. 
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self # 객체를 통해 호출하지 않으면 디스크립터 자신을 반환한다.
        else:
            return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be 0')

In [10]:
LineItem.price

In [11]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

< 참고 > 프로퍼티 팩토리와 디스크립터 클래스
예제 19-24에서 본 프로퍼티 팩토리에 몇 줄 추가함으로써 디스크립터 클래스를 향상시키는 것도 어렵지 않다. \_\_counter 변수를 팩토리 함수 객체 자체 속성으로 정의하면 팩토리의 실행이 끝나더라도 그 상태를 영구 보존할 수 있다. 그러나 다음과 같은 이유로 디스크립터 클래스를 추천한다.
+ 디스크립터 클래스는 상속을 이용해서 확장할 수 있다. 팩토리 함수는 재사용이 힘들다.
+ 함수 속성과 클로저에 상태를 저장하는 것보다 클래스와 객체 속성에 저장하는 것이 더 간단하다.


In [12]:
""" [예제 20-5] bulkfood_v4prop.py: 예제 20-2와 기능은 동일하지만, 디스크립터 클래스 대신 프로퍼티 팩토리를 사용한 예 """
def quantity(): # storage_name 인수가 없다.
    try: 
        quantity.counter += 1 # 카운터 상태를 보존하기 위한 클래스 속성에 의존할 수 없으므로 quantity() 함수 자체의 속성으로 정의한다.
    except:
        quantity.counter = 0  # quantity.counter가 정의되어 있지 않으면 0으로 설정한다.
        
    storage_name = '_{}:{}'.format('quantity', quantity.counter) # 객체 속성이 없으므로 storage_name을 지역 변수로 설정하고, 클로저를 이용해서
                                                                 # 나중에 qty_getter와 qty_setter가 사용할 수 있게 유지한다.
    def qty_getter(instance):
        return getattr(instance, storage_name)
    
    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, gty_setter)

#### 20.1.3 LineItem 버전 #5: 새로운 디스크립터 형
디스크립터를 상속하여 기능을 확장하는 방법을 알아보자.
<dl>
    <p>
      <dt><b>AutoStorage</b></dt>
      <dd>저장소 속성을 자동으로 관리하는 디스크립터 클래스</dd>
    </p>
    <p>
      <dt><b>Validated</b></dt>
      <dd>__set__() 메서드를 오버라이드해서 서브클래스에 반드시 구현해야 하는 validate()를 호출하는 AutoStorage의 추상 서브클래스</dd>
    </p>
</dl>

In [13]:
import abc

class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value) # 검증 기능은 상속받은 클래스에 구현되어 있으며 상위 클래스 AutoStroage는 저장만 담당한다.

        
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value): 
        value = self.validate(instance, value) # validate 함수에 검증을 위임한다. 
        super().__set__(instance, value) # 검증 후 반환된 값을 이용해서 실제로 값을 저장하는 슈퍼클래스의 __set__() 메서드를 호출한다.
        
    @abc.abstractmethod
    def validate(self, instance, value):
        """ 검증된 값을 반환하거나 ValueError를 발생시킨다. """

        
class Quantity(Validated):
    """ 양수임을 검증 """
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("value must be > 0")
        else:
            return value # 구상 메서드가 검증된 값을 반환하도록 구현하여 전달받은 데이터를 정리, 변환
        
        
class NonBlank(Validated):
    """ 최소 하나 이상의 비공백 문자가 들어있는 문자열 """
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [14]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [15]:
notebook = LineItem('', 1.2, 120) # description을 꼭 적어야 함

ValueError: value cannot be empty or blank

### 20.2 오버라이딩 디스크립터와 논오버라이딩 디스크립터

\_\_set\_\_() 메서드의 정의 여부에 따라 두 가지 범주의 디스크립터를 생성한다. 서로 다른 작동 방식을 관찰하기 위해 아래 클래스를 테스트베드로 사용해서 디스크립터의 작동을 살펴본다.